In [5]:
import requests

In [6]:
baseURL = 'https://explorer.natureserve.org/'
apiURL = 'api/data/taxon/102187'
response = requests.get('https://api.gbif.org/v1/species')
response

<Response [200]>

In [7]:
response.json()

{'offset': 0,
 'limit': 20,
 'endOfRecords': False,
 'results': [{'key': 0,
   'nubKey': 0,
   'nameKey': 58639884,
   'taxonID': 'gbif:0',
   'sourceTaxonKey': 0,
   'kingdom': 'incertae sedis',
   'kingdomKey': 0,
   'datasetKey': 'd7dddbf4-2cf0-4f39-9b2a-bb099caae36c',
   'constituentKey': 'd7dddbf4-2cf0-4f39-9b2a-bb099caae36c',
   'scientificName': 'incertae sedis',
   'canonicalName': 'incertae sedis',
   'authorship': '',
   'nameType': 'PLACEHOLDER',
   'rank': 'KINGDOM',
   'origin': 'SOURCE',
   'taxonomicStatus': 'DOUBTFUL',
   'nomenclaturalStatus': [],
   'remarks': '',
   'numDescendants': 2300,
   'lastCrawled': '2021-09-14T11:18:11.287+0000',
   'lastInterpreted': '2021-09-14T11:15:55.543+0000',
   'issues': [],
   'synonym': False},
  {'key': 1,
   'nubKey': 1,
   'nameKey': 58512980,
   'taxonID': 'gbif:1',
   'sourceTaxonKey': 1,
   'kingdom': 'Animalia',
   'kingdomKey': 1,
   'datasetKey': 'd7dddbf4-2cf0-4f39-9b2a-bb099caae36c',
   'constituentKey': 'd7dddbf4-2cf0-4

# Leaderboards and Ranking

In [8]:
## Imports
import pymysql
import pandas as pd
from datetime import datetime, date
import numpy as np

### MySQL Connection

In [9]:
cnxn = pymysql.connect(
            host='wildlife-tagging.c1df6qqn09wc.us-east-2.rds.amazonaws.com',
            user='admin',
            password = "TaggingIsFun!1128",
            port=3306,
            db='wildlife-tagging',
            charset='utf8'
            )
cursor = cnxn.cursor()

In [10]:
query = 'SELECT UUID, Tagger_name FROM Tagger'
tagger_df = pd.read_sql(query, cnxn)
query = 'SELECT UUID, Researcher_name FROM Researcher'
researcher_df = pd.read_sql(query, cnxn)
# # Used for testing the leaderboard sorting so some taggers have multiple entries
# tagger_df = tagger_df.append(df.iloc[0])

In [11]:
'''
 create_leaderboard, returns DataFrame
    Returns the top 10 IDs sorted by frequency. 
    Converts IDs to names to increase readibility.

 Parameters
    df = DataFrame with data
    id = Column name that holds id
    name = Column name that holds the names that the id maps too
    top = Top values to return, default: 10
'''
def create_leaderboard(df, id:str, name:str, top = 10):
    nameMap = df.drop_duplicates()
    nameMap = nameMap.set_index(id)[name].squeeze()

    df_grouped = df.groupby([id]).count().reset_index()
    df_grouped.columns = ['Name', 'Count']
    df_grouped['Name'] = df_grouped['Name'].map(nameMap)

    leaderboard = df_grouped.sort_values(by='Count', ascending=False).head(top)

    # This line is only necessary to make the index start from 1 so first place is labeled as #1 and not #0
    leaderboard.index += 1 
    return leaderboard

In [12]:
create_leaderboard(tagger_df, 'UUID', 'Tagger_name')


,Name,Count
1,Andrew,1
2,Cammie Rigo,1
3,Caprice Chapdelaine,1
4,Rowena Mose,1
5,Carmen Storms,1
6,Wynell Ortez,1
7,Camie Whisenhunt,1


In [13]:
create_leaderboard(researcher_df, 'UUID', 'Researcher_name')


,Name,Count
1,Steven Sampson,1
2,Jill Adams,1
3,Bob Jones,1


## Get Current User Stats

In [14]:
def get_current_user_statistics(user_uid):
#user_uid = "apovkYcqoQPFwzvojf8u7CqdZ2s1"

    total_img_tagged = 0
    today_img_tagged = 0
    leaderboard_rank = "N.A."
    target_status = "Incomplete"
    user_level = "Beginner"

    try:
        cursor.execute(f"""
        SELECT * FROM 
            (SELECT Tagger_UUID, COUNT(*) as Total_Tagged, RANK()
            OVER ( order by COUNT(*) desc ) Leaderboard_Rank 
            FROM Tag_Info 
            GROUP BY Tagger_UUID) as temp
        WHERE Tagger_UUID = '{user_uid}'
        """)

        result = [i for i in cursor.fetchone()]
        total_img_tagged, leaderboard_rank = result[1], result[2]
    except Exception as e:
        print(e)

    try:
        cursor.execute(f"""
        SELECT Tagger_UUID, COUNT(*) as Today_Tagged FROM Tag_Info 
        WHERE Tagged_Date = '{date.today()}' and Tagger_UUID = '{user_uid}' 
        GROUP BY Tagger_UUID
        """)

        result = [i for i in cursor.fetchone()]
        today_img_tagged = result[1]
    except Exception as e:
        print(e)

    try:
        cursor.execute(f"SELECT Level FROM Tagger WHERE UUID='{user_uid}'")

        result = [i for i in cursor.fetchone()]
        user_level = result[0]
    except Exception as e:
        print(e)
        
    return {
        "total_img_tagged": total_img_tagged,
        "today_img_tagged": today_img_tagged,
        "leaderboard_rank": leaderboard_rank,
        "target_status": target_status,
        "user_level": user_level
    }

In [15]:
get_current_user_statistics("apovkYcqoQPFwzvojf8u7CqdZ2s1")

'NoneType' object is not iterable


{'total_img_tagged': 4,
 'today_img_tagged': 0,
 'leaderboard_rank': 3,
 'target_status': 'Incomplete',
 'user_level': 'Advanced'}

In [16]:
get_current_user_statistics("r9IhDY79cOZ8ai8Au8wcrS551Vg2")

'NoneType' object is not iterable


{'total_img_tagged': 10,
 'today_img_tagged': 0,
 'leaderboard_rank': 1,
 'target_status': 'Incomplete',
 'user_level': 'Beginner'}

In [17]:
get_current_user_statistics("HRXzDx4aaoabECYhD33gNB8H7Gv1")

'NoneType' object is not iterable


{'total_img_tagged': 5,
 'today_img_tagged': 0,
 'leaderboard_rank': 2,
 'target_status': 'Incomplete',
 'user_level': 'Advanced'}

## Leaderboards Generator

In [18]:
user_uid = "apovkYcqoQPFwzvojf8u7CqdZ2s1"

lb_df = pd.read_sql(f"""
        SELECT * FROM Tagger Left Outer Join 
            (SELECT Tagger_UUID, MAX(Tagged_Date) as Last_Active, COUNT(*) as Total_Tagged, RANK()
            OVER ( order by COUNT(*) desc ) Leaderboard_Rank 
            FROM Tag_Info 
            GROUP BY Tagger_UUID) as temp
            on UUID=Tagger_UUID
        ORDER BY -Leaderboard_Rank DESC
        LIMIT 25
        """, con=cnxn)

lb_df.drop(['UUID', 'Tagger_UUID'], axis=1, inplace=True)
lb_df['Last_Active'] = (pd.to_datetime(lb_df['Last_Active'])).dt.strftime("%b %d, %Y")
lb_df.fillna("-", inplace=True)
lb_df

,Tagger_Name,Email_Address,Organization,Level,Last_Active,Total_Tagged,Leaderboard_Rank
0,Wynell Ortez,ortez@test.com,Taggers Inc.,Beginner,"Oct 10, 2021",10.0,1.0
1,Caprice Chapdelaine,cc@test.com,Taggers Inc.,Advanced,"Oct 10, 2021",5.0,2.0
2,Rowena Mose,mose@test.com,Georgia Institute of Technology,Advanced,"Oct 06, 2021",4.0,3.0
3,Camie Whisenhunt,ap@test.com,Georgia Institute of Technology,Beginner,"Oct 10, 2021",3.0,4.0
4,Andrew,aibauer7@gmail.com,Georgia Institute of Technology,Beginner,-,-,-
5,Cammie Rigo,cr@test.com,Bio Utd.,Advanced,-,-,-
6,Carmen Storms,stormz@test.com,Bio Utd.,Beginner,-,-,-


In [19]:
# This function is only for testing due to there being no AnimalID's in the db
def add_IDS(tag_info_df): 
    tag_info_df['Animal_ID'] = np.random.randint(1, 3, tag_info_df.shape[0])
    return tag_info_df

def pull_sql_data():
    query = 'SELECT * FROM Image'
    image_df = pd.read_sql(query, cnxn)
    query = 'SELECT * FROM Tag_Info'
    tag_info_df = pd.read_sql(query, cnxn)
    query = 'SELECT * FROM Animal'
    animal_df = pd.read_sql(query, cnxn)
    query = 'SELECT * FROM Researcher'
    researcher_df = pd.read_sql(query, cnxn)

    # Minor adjustments
    image_df = image_df.rename(columns = {'ImageID':'Image_ID'})
    
    # Animal_ID 0 has multiple rows 
    animal_df = animal_df.drop_duplicates(subset='AnimalID')

    # # Just for testing, should be unnecessary when animal_id column is populated
    # tag_info_df = add_IDS(tag_info_df)

    return image_df, tag_info_df, animal_df, researcher_df

def join_tag_img(tag_info_df, image_df):
    # Convert date columns to datetime objects
    image_df['UploadDate'] =  pd.to_datetime(image_df['UploadDate'])
    image_df['CaptureDate'] = pd.to_datetime(image_df['CaptureDate'])
    tag_info_df['Tagged_Date'] = pd.to_datetime(tag_info_df['Tagged_Date'])

    # Join both dataframes based on the Image_ID
    tag_info_df['Image_ID'] = tag_info_df['Image_ID'].astype('int64')
    joined_df = image_df.join(tag_info_df.set_index('Image_ID'), on='Image_ID')
    return joined_df

def mostCommon(df):
    counts = df.value_counts()
    if len(counts) > 0:
        return counts.idxmax()
    return None

def secondMostCommon(df):
    counts = df.value_counts()
    if len(counts) > 1:
        return counts.index[1]
    return None

def createImageSummary(startDate, endDate):
    image_df, tag_info_df, animal_df, researcher_df = pull_sql_data()
    df = join_tag_img(tag_info_df, image_df)
    
    # Filtering out dates that are out of range
    df = df[(df['UploadDate'] >= startDate) & (df['UploadDate'] <= endDate)]

    # Mapping ids to names from other dfs
    df['Common_Name'] = df.Animal_ID.map(animal_df.set_index('AnimalID').Animal_Common_Names)
    df['Scientific_Name'] = df.Animal_ID.map(animal_df.set_index('AnimalID').Animal_Scientific_Name)
    df['Common_Name'] = df['Common_Name'].str.split(",").str[0]
    df['Researcher_Name'] = df.ResearcherUUID.map(researcher_df.set_index('UUID').Researcher_Name)
    
    # Renaming Columns
    df['Tag_Count'] = df['Tag_ID']
    df['Most_Tagged_Animal'] = df['Common_Name']
    df['Second_Most_Tagged_Animal'] = df['Common_Name']
    df['Most_Tagged_Type'] = df['Animal_Type']
    df['Second_Most_Tagged_Type'] = df['Animal_Type']
    df['Animal_Present'] = (df['Animal_Present'] == "True").astype(int)
    
    groups = df.groupby('Image_ID')
    groups = groups.agg({
        'UploadDate': np.max,
        'CaptureDate': np.max,
        'Researcher_Name': np.max,
        'Location': np.max,
        'Tag_Count': pd.Series.count,
        'Animal_Present': np.average, # Returns percentage of true responses
        'Most_Tagged_Type': mostCommon,
        'Second_Most_Tagged_Type': secondMostCommon,
        'Most_Tagged_Animal': mostCommon,
        'Second_Most_Tagged_Animal': secondMostCommon,
        'Scientific_Name': mostCommon,
    })
    return groups.apply(lambda a: a[:])

test_start = datetime(2021, 5, 8)
test_end = datetime.today()
print(createImageSummary(test_start, test_end))

         UploadDate CaptureDate Researcher_Name  \
Image_ID                                          
4        2021-08-06  2021-08-06  Steven Sampson   
6        2021-05-21  2021-05-21  Steven Sampson   
7        2021-09-21  2021-09-21       Bob Jones   
9        2021-07-12  2021-07-12  Steven Sampson   

                                                   Location  Tag_Count  \
Image_ID                                                                 
4         https://animalsbucket.s3.us-east-2.amazonaws.c...          2   
6         https://animalsbucket.s3.us-east-2.amazonaws.c...          3   
7         https://animalsbucket.s3.us-east-2.amazonaws.c...          2   
9         https://animalsbucket.s3.us-east-2.amazonaws.c...          2   

          Animal_Present Most_Tagged_Type Second_Most_Tagged_Type  \
Image_ID                                                            
4                    1.0           Mammal                    None   
6                    1.0           Mammal